# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk import word_tokenize
import gzip
import pickle
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download(['punkt', 'wordnet'])

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from sklearn.metrics import precision_score
#print(classification_report(y_test, y_pred, target_names=category_names))
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV # For optimization
from sklearn.metrics import classification_report,confusion_matrix, precision_score,\
recall_score,accuracy_score,  f1_score,  make_scorer

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * from Message", engine)
X = df.message
Y = df.loc[:,"related":"direct_report"]
category_names = Y.columns

In [3]:
df

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
8,8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1


In [4]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [5]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
     # Normalize text
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    
    
     # Tokenize text
    words = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    
    # Remove stop words
    tokens = [w for w in words if w not in stopwords.words('english')]  

    
    # iterate through each token
    cleanTokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() # [WordNetLemmatizer().lemmatize(w) for w in tokens]
        cleanTokens.append(clean_tok)
    
    return cleanTokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
print('Training set score: ', pipeline.score(X_train,y_train))
print('Test set score: ' ,pipeline.score(X_test,y_test))

Training set score:  0.816905706439
Test set score:  0.238327738785


In [11]:
y_pred = pipeline.predict(X_test)
#print(classification_report(y_test, y_pred, target_names=category_names))
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [12]:
accuracy = (y_pred == y_test).mean()
print("Accuracy:", accuracy)

Accuracy: related                   0.805005
request                   0.885413
offer                     0.994660
aid_related               0.759231
medical_help              0.922490
medical_products          0.949344
search_and_rescue         0.973756
security                  0.980470
military                  0.967196
child_alone               1.000000
water                     0.960635
food                      0.924626
shelter                   0.931950
clothing                  0.986878
money                     0.978029
missing_people            0.988251
refugees                  0.970400
death                     0.955294
other_aid                 0.869698
infrastructure_related    0.933323
transport                 0.955600
buildings                 0.954074
electricity               0.979707
tools                     0.992981
hospitals                 0.987946
shops                     0.995117
aid_centers               0.987946
other_infrastructure      0.956820
weather_re

In [13]:
# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer() 
clf = RandomForestClassifier()  # MultinomialNB()

# Fit and/or transform each to the data
vect_count = vect.fit_transform(X_train)
X_train_tf = tfidf.fit_transform(vect_count)
clf.fit(X_train_tf, y_train )  #tfidf.fit_transform(vect.fit_transform(X_test))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
# Transform test data
X_test_count = vect.transform(X_test)
X_test_tf = tfidf.transform(X_test_count)


# Predict test labels
y_pred = clf.predict(X_test_tf)

In [15]:
#y_pred.columns
y_pred_df = pd.DataFrame(y_pred)

In [16]:
type_of_target(y_test)
type_of_target(y_pred)
y_pred_df = pd.DataFrame(y_pred)
#print(classification_report(np.hstack(y_test.values.astype(int)), np.hstack(y_pred_df.values.astype(int)), target_names = df.columns[4:]))  #, target_names=target_names

In [17]:
for i,col in enumerate(category_names):
        print(col)
        print(classification_report(np.hstack(y_test.values.astype(int)), np.hstack(y_pred_df.values.astype(int))))
        print("***\n")

related
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.57     20826
          2       0.23      0.42      0.30        45

avg / total       0.94      0.94      0.93    235944

***

request
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.57     20826
          2       0.23      0.42      0.30        45

avg / total       0.94      0.94      0.93    235944

***

offer
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.57     20826
          2       0.23      0.42      0.30        45

avg / total       0.94      0.94      0.93    235944

***

aid_related
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
def avg_accuracy(y_test, y_pred):
    """
    This is the score_func used in make_scorer, which would be used in in GridSearchCV 
    """
    avg_accuracy=accuracy_score(y_test.values.reshape(-1,1), y_pred.reshape(-1,1))
    
    return avg_accuracy
avg_accuracy_cv = make_scorer(avg_accuracy)


In [20]:
parameters = parameters = {
    #'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),  
    'clf__estimator__max_depth': [15, 30],  
    'clf__estimator__n_estimators': [100, 250]}

cv = GridSearchCV(
    pipeline, 
    param_grid=parameters,
    cv=3,
    scoring=avg_accuracy_cv, 
    verbose=3)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=100, score=0.9288644763164141, total= 1.9min
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=100 ..


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.9min remaining:    0.0s


[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=100, score=0.9289873868375547, total= 1.9min
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=100 ..


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.9min remaining:    0.0s


[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=100, score=0.9290848675957006, total= 1.9min
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=250 ..
[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=250, score=0.9287839487335978, total= 2.6min
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=250 ..
[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=250, score=0.9290128166005492, total= 2.6min
[CV] clf__estimator__max_depth=15, clf__estimator__n_estimators=250 ..
[CV]  clf__estimator__max_depth=15, clf__estimator__n_estimators=250, score=0.9289026209609059, total= 2.6min
[CV] clf__estimator__max_depth=30, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=30, clf__estimator__n_estimators=100, score=0.9332468721391517, total= 2.3min
[CV] clf__estimator__max_depth=30, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=30, clf__estimator__n_estimators=100, score=0.9336495100532329,

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 45.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [15, 30], 'clf__estimator__n_estimators': [100, 250]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(avg_accuracy), verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
cv.best_score_

0.93332457419274628

In [22]:
cv.best_params_

{'clf__estimator__max_depth': 30, 'clf__estimator__n_estimators': 100}

In [23]:
refined_pipeline = cv.best_estimator_
y_pred_best = refined_pipeline.predict(X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
for i,col in enumerate(category_names):
        print(col)
        print(classification_report(np.hstack(y_test.values.astype(int)), np.hstack(y_pred_df.values.astype(int))))
        print("***\n")

related
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.57     20826
          2       0.23      0.42      0.30        45

avg / total       0.94      0.94      0.93    235944

***

request
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.57     20826
          2       0.23      0.42      0.30        45

avg / total       0.94      0.94      0.93    235944

***

offer
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.57     20826
          2       0.23      0.42      0.30        45

avg / total       0.94      0.94      0.93    235944

***

aid_related
             precision    recall  f1-score   support

          0       0.95      0.99      0.97    215073
          1       0.83      0.43      0.

### 9. Export your model as a pickle file

In [26]:
filename = 'classifier.gzip'
pickle.dump(pipeline, open(filename, 'wb'))

In [29]:
pickle.dump(pipeline, open("classifier.pkl", "wb"))

In [27]:
def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.